## BERT interpretability

In [1]:
import os
import sys
import torch
import captum
import random
import numpy as np

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)


from datasets import BERTDeepLNarrativeDataset
from models.bert import BERT

In [2]:
# Create datasets
data_paths = {
    'main': os.path.join(parent_dir, 'data', 'EN')
}

datasets = {}

for topic in ['UA', 'CC']:
    for split in ['train', 'val', 'dev']:
        datasets[f'{topic}_{split}'] = BERTDeepLNarrativeDataset(
            data_paths=data_paths,
            topic=topic,
            split=split,
            max_length=512,
            val_split=0.2,
        )


In [3]:
# Create model for a specific topic
topic = 'CC'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERT(
    id2label=datasets[f'{topic}_train'].index2label,
    finetune=True
)

# Load state dict
state_dict = torch.load(
    os.path.join(parent_dir, 'trained_models', f'bert_{topic}_best.pt')
)

model.load_state_dict(state_dict)
model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=

In [19]:
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization
import torch.nn.functional as F


def tokenize_input(tokens):
    return [datasets[f'{topic}_train'].tokenizer._convert_token_to_id(t) for t in tokens]

def decode_input_ids(ids):
    return [datasets[f'{topic}_train'].tokenizer._convert_id_to_token(i.item()) for i in ids]

def predict(inputs, attention_mask=None, token_type_ids=None):
    preds = model(inputs, attention_mask, token_type_ids)
    return F.softmax(preds, dim=-1)

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

def visualize_attributions(inputs, labels):
    count_special_tokens = np.isin(inputs['input_ids'], [0, 101, 102]).sum()
    count_pads = count_special_tokens - 2 # Subtract 2 for [CLS] and [SEP] (padding is at the end)

    ref_input_ids = torch.tensor(
        tokenize_input(['[CLS]'] + ['[PAD]'] * (512 - count_special_tokens) + ['[SEP]'] + ['[PAD]'] * count_pads), dtype=torch.long, device=device
    ).unsqueeze(0)

    inputs = {k: v.to(device) for k, v in inputs.items()}
    pred_prob = predict(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])[0]
    actual_labels = [datasets[f'{topic}_train'].index2label[i] for i, l in enumerate(labels) if l]

    threshold = 0.20
    predicted_labels = [datasets[f'{topic}_train'].index2label[i] for i, l in enumerate(pred_prob >= threshold) if l]
    tokenized_text = decode_input_ids(inputs['input_ids'][0])

    print(f'Actual: {actual_labels}')
    print(f'Predicted: {predicted_labels}')

    lig = LayerIntegratedGradients(
        model,
        model.model.bert.embeddings.word_embeddings,
    )

    viz_records = []

    # Compute attributions for each true positive class and a random negative class
    classes = [i for i, l in enumerate(labels) if l] + random.sample([i for i, l in enumerate(labels) if not l], 1)

    for class_idx in classes:

        pred_prob_class = pred_prob[class_idx]

        attributions, delta = lig.attribute(
            inputs=inputs['input_ids'],
            baselines=ref_input_ids,
            additional_forward_args=(inputs['attention_mask'], inputs['token_type_ids']),
            target=class_idx,
            return_convergence_delta=True,
            n_steps = 50
        )

        agg_attributions = summarize_attributions(attributions[0])

        viz_record = visualization.VisualizationDataRecord(
            word_attributions=agg_attributions.tolist(),
            pred_prob=pred_prob_class.item(),
            pred_class=', '.join(predicted_labels),
            true_class=', '.join(actual_labels),
            attr_class=datasets[f'{topic}_train'].index2label[class_idx],
            attr_score=agg_attributions.sum(),
            raw_input_ids=tokenized_text,
            convergence_score=delta[0]
        )
        viz_records.append(viz_record)

    visualization.visualize_text(viz_records)

In [24]:
index = 17

# Create input
inputs, labels, doc_name = datasets[f'{topic}_train'][index]
print(doc_name)

# Visualize attributions
visualize_attributions(inputs, labels)

EN_CC_200116.txt
Actual: ['CC: Criticism of climate movement', 'CC: Criticism of climate policies', 'CC: Criticism of institutions and authorities', 'CC: Criticism of climate movement: Climate movement is alarmist', 'CC: Criticism of climate policies: Climate policies have negative impact on the economy', 'CC: Criticism of climate policies: Other', 'CC: Criticism of institutions and authorities: Criticism of political organizations and figures']
Predicted: ['CC: Criticism of institutions and authorities']


### Qualitative Analysis of the CC Topic:

#### 1. CC: Criticism of climate movement:
This label focuses on critiques of the climate movement being alarmist. The green words emphasize negative sentiment around "climate alarmism" and terms like "alarmist" and "money," which relate to the criticism theme. The red words like "uraine" and "trudeau" minimize the focus, indicating overlap with geopolitical and institutional criticism.

#### 2. CC: Criticism of climate policies:
The label closely aligns with criticisms of climate policies, particularly the "carbon tax." This issue is highlighted by words such as "tax," "policies," and "mis ##guide". The overlap with geopolitical terminology ("ur ##aine") implies narrative connection but does not dominate.

#### 3.CC: Criticism of institutions and authorities
We can see alignment with critiques of institutional leadership, notably Justin Trudeau. Words like "minister," "speech," "mis ##guide," and "federal" substantiate this. There’s strong attribution to criticisms of leadership and institutions within the climate narrative.

#### 4. CC: Criticism of climate movement: Climate movement is alarmist
This sub-label narrows criticism to the alarmist nature of the climate movement. Words like "climate alarm ##ism" strongly support this, but the inclusion of "nazi fighter" and geopolitical context distracts.

#### 5. CC: Criticism of climate policies: Climate policies have negative impact on the economy
The sub-label focuses criticism on economic impacts, particularly the "carbon tax." The green words clearly outline financial concerns, with "the economy" and "social crisis" providing context. Red words minimally distract.

#### 6. CC: Criticism of climate policies: Other
This label broadens the critique of climate policies beyond economic impacts and remains relevant to policy critique.

#### 7. CC: Criticism of institutions and authorities: Criticism of political organizations and figures
This sub-label has high attribution to critiques of political figures, particularly Trudeau. The green words effectively highlight institutional criticism with minimal distractions.

#### 8. CC: Criticism of climate movement: Other
We can observe the generalized criticism of the climate movement. Attribution to words like "climate alarm ##ism" and "economy" aligns with the theme, but geopolitical overlap adds slight noise.

**Conclusion**:
The model's prediction is reasonable given the overlap in themes among the true labels. However, the high attribution scores for alternative labels suggest that the model might benefit from more nuanced differentiation between similar labels. Refining the data or adding more context-specific examples might improve the classification accuracy.




In [25]:
# Create model for a specific topic
topic = 'UA'
model = BERT(
    id2label=datasets[f'{topic}_train'].index2label,
    finetune=True
)

# Load state dict
state_dict = torch.load(
    os.path.join(parent_dir, 'trained_models', f'bert_{topic}_best.pt')
)

model.load_state_dict(state_dict)
model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=

In [43]:
index = 6

# Create input
inputs, labels, doc_name = datasets[f'{topic}_train'][index]
print(doc_name)

# Visualize attributions
visualize_attributions(inputs, labels)

EN_UA_023816.txt
Actual: ['URW: Blaming the war on others rather than the invader', 'URW: Discrediting Ukraine', 'URW: Discrediting the West, Diplomacy', 'URW: Russia is the Victim', 'URW: Blaming the war on others rather than the invader: The West are the aggressors', 'URW: Discrediting Ukraine: Ukraine is a puppet of the West', 'URW: Discrediting the West, Diplomacy: Other', 'URW: Russia is the Victim: The West is russophobic', 'URW: Russia is the Victim: UA is anti-RU extremists']
Predicted: ['URW: Discrediting the West, Diplomacy']


### Qualitative Analysis of the URW Topic: 

#### 1. URW: Blaming the war on others rather than the invader
The focus on external actors' alleged involvement aligns strongly with this narrative. The high score reflects its relevance to the narrative that others are responsible for the conflict. Words like "troops," "Russian," "Polish," "armed forces," "NATO," "military personnel fighting," "proxy," and "geopolitical domination" support the narrative of blaming external actors, particularly NATO and Western countries, for the war. Words like "Washington" and "today" detract from this label.

#### 2. URW: Discrediting Ukraine
This label reflects a narrative of Ukraine being portrayed as weak or a puppet, with the green words emphasizing dependency on the West. Words like "mercenaries," "forces," "fighting," "proxy," "nationalist sentiment," and "domination" align with the narrative of portraying Ukraine as dependent and a proxy.

#### 3. URW: Discrediting the West, Diplomacy
This label effectively captures the narrative of undermining Western diplomatic efforts and highlighting perceived ulterior motives. Words like "NATO," "geopolitical domination," "Western countries," and "proxy against" should align with criticizing Western involvement.

#### 4. URW: Russia is the Victim
The narrative portrays Russia as being unjustly targeted, emphasizing external threats and geopolitical dominance. Words like "proxy," "Western," "nationalist sentiment," "domination," and "threats" strongly align with portraying Russia as being under attack.

#### 5. URW: Blaming the war on others rather than the invader: The West are the aggressors
This label reflects a focused narrative of Western aggression and is highly relevant, as shown by the strong attribution score. "proxy," "cannon fodder," "domination," "NATO," "important," and "most" are words strongly aligning with blaming Western aggression.

#### 6. URW: Discrediting Ukraine: Ukraine is a puppet of the West
The narrative concentrates on Ukraine's reliance on the West, which corresponds closely to green terms that emphasize manipulation and control. The terms "proxy," "defeat," "nationalist sentiment," and "puppet of the West" all fit this story well.

#### 7. URW: Discrediting the West, Diplomacy: Other
While aligned with general criticisms of the West, this label is less specific than others and reflects broader diplomatic narratives. Words such as "proxy," "nationalist sentiment," "domination," and "geopolitical" are associated with denouncing the West's influence.

#### 8. URW: Russia is the Victim: The West is Russophobic
This storyline is consistent with the text, emphasizing the themes of Western antagonism and Russophobia. "proxy," "NATO," "domination," and "threats" are, again, words that fit the narrative of Western hostility towards Russia.

#### 9. URW: Russia is the Victim: UA is anti-RU extremists
The attribution score indicates weaker alignment, as the narrative about extremist behavior is less supported by the text. Words such as "proxy," "Western," and "domination" partially align with this narrative and words like "anti," "sentiment," and "important" reduce alignment.

#### 10. URW: Praise of Russia: Russian invasion has strong national support
This label is not supported by the text, with the negative attribution score reflecting misalignment with the narrative of praising Russia. We have words like "nationalist sentiment" and "support", which partially align and words like "anti-Russia," "cannon fodder," and "domination", which heavily detract.

**Conclusion**:
The analysis highlights that the model struggles to accurately predict narratives when multiple labels overlap significantly, underscoring the complexity of distinguishing nuanced propaganda themes in text. This could suggest room for improvement in fine-tuning or refining the attribution-based classification process.